In [7]:
import os

# Tool Creation

Create directory for tool

In [8]:
tool_folder_path = "../../src/tools/pharmgkb"
os.makedirs(tool_folder_path, exist_ok=True)

Create init file

In [9]:
init_file_path = os.path.join(tool_folder_path, "__init__.py")

init_file_content = """# Auto-generated package marker.
"""

with open(init_file_path, "w", encoding="utf-8") as f:
    f.write(init_file_content)


Create main node file with desired functionality

In [10]:
node_file_path = os.path.join(tool_folder_path, "node.py")

node_file_content = """

from __future__ import annotations

import time

from src.state import State
from src.tools.base import Node
from typing import List
import requests

DEBUG = True

def pharmgkb_agent(state: "State") -> "State":
    variant_entities = state.get("variant_entities") or {}

    # Iterate through gene entities saved in the state
    for variant_name, variant in variant_entities.items():
        if not variant_name:
            continue

        summary_lines: List[str] = []

        try:
            url = "https://api.clinpgx.org/v1/data/variantAnnotation"
            params = {
                "location.fingerprint": variant_entities,
                "view": "max"
            }

            response = requests.get(url, params=params)

            descriptions = []
            if response.ok:
                data = response.json()
                for i in data['data']:
                    descriptions.append(i['description']) 
            else:
                print("Error:", response.text)

            if descriptions:
                summary = f"Descriptions of specific variant-drug or variant-phenotype associations annotated to {variant} according to PharmGKB: {', '.join(descriptions)}."
                summary_lines = f"*PharmGKB: {summary}"
                variant.update_text_summaries(summary_lines)
                variant.add_tool("PharmGKB")

        except Exception as e:
            print(variant_name, e)

        time.sleep(0.3)  # courteous pause

    if DEBUG:
        print(f"[PharmGKB] Predictions fetched")

    return 

# Specify name and detailed description of tool
NODES: tuple[Node, ...] = (
    Node(
        name="PharmGKB",
        entry_point=pharmgkb_agent,
        description="Fetches information about variant associations to drugs and phenotypes manually curated from literature through PharmGKB.",
    ),
)
"""

with open(node_file_path, "w", encoding="utf-8") as f:
    f.write(node_file_content)

# Use Case Example

Enter a question into Alvessa's user interface

<img src="imgs/question.png" width="800"/>

Alvessa will now be able to pull from the information called by the new Cosmic tool to answer the user's questions

<img src="imgs/answer.png" width="800"/>